### Create the below table in Azure SQL DB before proceeding with the below steps

<p>   CREATE TABLE CUSTOMER 
        (
<br>    C_CUSTKEY		INT,
<br>	C_NAME			VARCHAR(25),
<br>	C_ADDRESS		VARCHAR(40),
<br>	C_NATIONKEY		SMALLINT,
<br>	C_PHONE			CHAR(15),
<br>	C_ACCTBAL		DECIMAL,
<br>	C_MKTSEGMENT	CHAR(10),
<br>	C_COMMENT		VARCHAR(117)
        );</p>

In [ ]:
from pyspark.sql.types import *

In [ ]:
# Details about connection string
logicalServername = "devtestlogicalserver.database.windows.net"
databaseName = "demoDB"
tableName = "dbo.CUSTOMER"
userName = "sqluser"
password = "StrongPwd" # Please specify password here
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(logicalServername, 1433, databaseName)
connectionProperties = {
  "user" : userName,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


In [ ]:
#Creating a schema which can be passed while creating the dataframe
cust_schema = StructType([
    StructField("C_CUSTKEY", IntegerType()),
    StructField("C_NAME", StringType()),
    StructField("C_ADDRESS", StringType()),
    StructField("C_NATIONKEY", ShortType()),
    StructField("C_PHONE", StringType()),
    StructField("C_ACCTBAL", DecimalType(18,2)),
    StructField("C_MKTSEGMENT", StringType()),
    StructField("C_COMMENT", StringType())
])

In [ ]:
# Reading customer csv files in a dataframe. This Dataframe will be written to Customer table in Azure SQL DB
df_cust= spark.read.format("csv").option("header",True).schema(cust_schema).load("dbfs:/mnt/Gen2Source/Customer/csvFiles")

### Reading and writing data from and to Customers table using JDBC SQLServer Drivers

In [ ]:
#Reading the table we have created and we shouldn;t see any rows in the dataframe
df_jdbcRead= spark.read.jdbc(jdbcUrl,  
                   table=tableName, 
                   properties=connectionProperties)

display(df_jdbcRead.printSchema())

root
-- C_CUSTKEY: integer (nullable = true)
-- C_NAME: string (nullable = true)
-- C_ADDRESS: string (nullable = true)
-- C_NATIONKEY: short (nullable = true)
-- C_PHONE: string (nullable = true)
-- C_ACCTBAL: decimal(18,2) (nullable = true)
-- C_MKTSEGMENT: string (nullable = true)
-- C_COMMENT: string (nullable = true)

In [ ]:
#It shouldn't fetch any records as there is no data loaded into the table
display(df_jdbcRead.limit(5))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT


In [ ]:
# Writing the dataframe which has all the csv read from ADLS Gen-2 folder to Customer table in Azure SQL Db
df_cust.write.jdbc(jdbcUrl,  
                   mode ="append", 
                   table=tableName, 
                   properties=connectionProperties)

In [ ]:
#Lets read the table after loading the data to count the number of rows
df_jdbcRead= spark.read.jdbc(jdbcUrl,  
                   table=tableName, 
                   properties=connectionProperties)

#Counting number of rows
df_jdbcRead.count()

Out[18]: 150000

In [ ]:
display(df_jdbcRead.limit(5))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
15304,Customer#000015304,DtANXOH6c8AisOyFRhaOYMA,0,10-686-383-4283,7238.68,AUTOMOBILE,ckly special sentiments. close packages integrate doggedly inside the pending depos
1765,Customer#000001765,OrnBgRc3kbx9ukZnNiUM5pBPXyE3epagewVQBPZz,0,10-538-728-6104,7409.60,AUTOMOBILE,"final, special deposits sleep carefully furiously special deposits. evenly ironic p"
5500,Customer#000005500,rluGqUoaGezWcQDwMBmGR,0,10-931-168-1559,8037.76,MACHINERY,"ckly pending instructions cajole slyly carefully bold pinto beans. regular, thin t"
21746,Customer#000021746,RpI9QnqrN8spzXNiIHOW,0,10-961-333-3043,7425.55,MACHINERY,ffy packages. furiously express packages detect blithely furiously regular accounts. accounts haggle. bravely final
12874,Customer#000012874,"7MqIGEWlg 4jwK,WZIakKqzMa9XYbrZjW",0,10-592-120-9012,3521.63,HOUSEHOLD,to the regular deposits. idly ironic instructions a


### Reading and writing data from and to Customers table using Apache Spark connector for Azure SQL

In [ ]:
server_name = f"jdbc:sqlserver://{logicalServername}" 
database_name = "demoDB"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "dbo.Customer"
username = "sqladmin"
password = "Password@12345" # Please specify password here

### Reading data from Customers table using Apache Spark connector

###### Truncate the dbo.Customer table so that we are loading data into empty table.

In [ ]:
# Truncate the dbo.Customer table so that we are loading data into empty table.
sparkconnectorDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

In [ ]:
display(sparkconnectorDF.printSchema())

root
-- C_CUSTKEY: integer (nullable = true)
-- C_NAME: string (nullable = true)
-- C_ADDRESS: string (nullable = true)
-- C_NATIONKEY: short (nullable = true)
-- C_PHONE: string (nullable = true)
-- C_ACCTBAL: decimal(18,2) (nullable = true)
-- C_MKTSEGMENT: string (nullable = true)
-- C_COMMENT: string (nullable = true)

In [ ]:
display(sparkconnectorDF.limit(5))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT


In [ ]:
#This code is truncating the table and loading the data. If we need to preserve the indexes created in this table then when should use the .option("truncate",True) which will avoid dropping the index
try:
  df_cust.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("truncate",True) \
    .option("url", url) \
    .option("dbtable", tableName) \
    .option("user", userName) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("Connector write failed", error)

In [ ]:
display(sparkconnectorDF.limit(5))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
15304,Customer#000015304,DtANXOH6c8AisOyFRhaOYMA,0,10-686-383-4283,7238.68,AUTOMOBILE,ckly special sentiments. close packages integrate doggedly inside the pending depos
1765,Customer#000001765,OrnBgRc3kbx9ukZnNiUM5pBPXyE3epagewVQBPZz,0,10-538-728-6104,7409.60,AUTOMOBILE,"final, special deposits sleep carefully furiously special deposits. evenly ironic p"
24055,Customer#000024055,dkmvKLXgthM10est,0,10-443-728-9420,8234.65,FURNITURE,st the quickly unusual packages. carefully ironic ideas use furiously above the fin
15357,Customer#000015357,s QYzgTLZ3DR1mnGGH0qZnYlIdMuDxDeaDd235mI,0,10-253-483-2261,-569.03,BUILDING,ts snooze carefully among the ironic requests. blithely special platelets detect doggedly bold accounts. regu
21300,Customer#000021300,lv7hzz5M5 5MBtBnwv,0,10-427-367-9605,999.95,BUILDING,ckages wake slyly final accounts. carefully pending packages inside the furi


In [ ]:
#Appending records to the existing table
try:
  df_cust.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("append") \
    .option("url", url) \
    .option("dbtable", tableName) \
    .option("user", userName) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("Connector write failed", error)

In [ ]:
#Counting number of rows, it should be 300K.
df_jdbcRead.count()

Out[38]: 300000

In [ ]:
#Lets read the data from the table
sparkconnectorDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

In [ ]:
display(sparkconnectorDF.limit(10))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
142537,Customer#000142537,YvqyKqFXsRYVxZG600ZMTaoNgh,0,10-200-494-1102,148.81,BUILDING,longside of the instructions detect furiously quiet pinto beans. furiously
129911,Customer#000129911,3SJzijkRBAsjb2YHkrB,0,10-325-156-2671,8685.19,MACHINERY,sleep carefully. slyly final packages hinder slyly against the slyly bold courts. furiously unusual theodoli
129170,Customer#000129170,bUA9Ey56ZlcajLAuvnSJ,0,10-704-546-4142,6760.31,FURNITURE,apades print carefully against the slyly idle d
144117,Customer#000144117,"p9st93hyaiw9CHcY5HsnG,b,JkiYBjPzgDr5yGk",0,10-760-248-6009,1961.76,MACHINERY,"ate final, even excuses. specia"
129222,Customer#000129222,yGVvONgWF2mUKLmPGVMr5n4FhZ27vZH sO,0,10-435-165-5146,3420.06,BUILDING,ts cajole carefully about the
129344,Customer#000129344,"ppP,QJvSoa3WDl8EuDIF1N3xQOb1DX",0,10-207-325-4612,286.51,BUILDING,requests cajole quickly across the carefully
142829,Customer#000142829,2G5n98FAieUPnvO,0,10-413-250-6547,926.69,BUILDING,usly silent accounts wake slyly after the foxes. final pinto beans above the car
147100,Customer#000147100,"Kh,B7Yp8Of",0,10-844-528-2148,6242.81,FURNITURE,"ently express accounts eat theodolites. bold gifts after the regular, bold pinto beans detect about the quic"
135787,Customer#000135787,5ki7oerlqxNVtLafOQ0Z5 xLeJP5Z,0,10-227-900-4613,-175.42,HOUSEHOLD,according to the furiously regular instructions eat carefully alongside of the ironic deposits. furiously iro
146033,Customer#000146033,GFRV8dhZEnaqmvfA,0,10-779-727-4905,-661.34,FURNITURE,ironic deposits. final foxes play fluffily ironic accounts. carefully bold r
